Imports

In [118]:
import supplyactor #Ensemble des classes qui définissent les acteurs de la grande distribution
import numpy as np
from tqdm import tqdm

Paramètres 

In [46]:
n = 50 #nombre de magasin
supplyactor.SurStock = 1.001 #surstock en poucentage qu'effectuent les magasisns à chaque commande par rapport à leur prévisions 
VarTM = 5 #variation de la taille des magasins 


Scénario 1

In [47]:
#instanciation des magasins et des clients 
Magasin = []
for i in range(n):
    Magasin.append(supplyactor.Supermarket(1+VarTM*np.random.random(),"marque"+str(i),i))

for magasin in Magasin : 
    magasin.clients.append(supplyactor.Customer(10*magasin.taille,2*magasin.taille))

In [48]:
#initialisation des process ARMA pour les clients
for i in range(10*supplyactor.grande_période) : 
    for magasin in Magasin : 
        for client in magasin.clients : 
            client.ARMA()

#initialisation des prédictions pour les magasins
for i in range(10*supplyactor.grande_période) : 
    for magasin in Magasin : 
        magasin.EnregistrementDemande()
        for client in magasin.clients : 
            client.ARMA()

#première commande 
for magasin in Magasin : 
    magasin.Commande()

In [50]:
#simulation 
for i in tqdm(range(100*supplyactor.grande_période)):
    
    supplyactor.temps()#incrément de temps 

    for magasin in Magasin:
        magasin.Inventaire() #état des stocks
        magasin.Vente() #ventes aux clients des produits disponible en fonction des demandes de leur clients 
        magasin.EnregistrementDemande() #enregistrement de la demande des clients pour les commandes futures 

        for client in magasin.clients : 
            client.ARMA() #actualisation de la demade des clients 

        if i % supplyactor.grande_période == 0:
            magasin.Inventaire() #état des stocks
            magasin.Commande() #commande aux suppliers

In [51]:
#Collecte des résultats

#Denrées perdues 
Gaspillage = 0
for magasin in Magasin : 
    x = magasin.Rebut()
    print(x)
    for product in x.keys() :
        Gaspillage += x[product]

#Demandes non satisfaites 
Pénurie = 0
for magasin in Magasin : 
    Pénurie += magasin.quantitéRupture 

#total des ventes 
Ventes = 0
for magasin in Magasin : 
    Ventes += magasin.ventes

#enregistrement
Scénario1 = {"Pénurie" : Pénurie, "Gaspillage":Gaspillage,"Ventes":Ventes}

{'lait': 628.5458111850652, 'oeuf': 479.3200046839829, 'beurre': 86.4028303003633, 'crème': 75.66556898960042, 'fromage': 135.99984426156905, 'Goûter': 61.35504293246754, 'Fruit': 152.73800609532483, 'légumes': 463.157721861087}
{'lait': 176.21948233345387, 'oeuf': 34.320086859604665, 'beurre': 231.6929554140668, 'crème': 67.44776451622478, 'fromage': 86.75116179760349, 'Goûter': 111.4052917052594, 'Fruit': 57.83740743438318, 'légumes': 62.07786730443838}
{'lait': 67.42238813307678, 'oeuf': 120.862249071387, 'beurre': 111.51265295336428, 'crème': 155.9191987091412, 'fromage': 312.79059039053953, 'Goûter': 159.84697641766482, 'Fruit': 163.6409336416507, 'légumes': 69.35071641614684}
{'lait': 129.16247605253244, 'oeuf': 131.92618444712556, 'beurre': 101.44487283873542, 'crème': 123.33193600822611, 'fromage': 96.7359823424475, 'Goûter': 83.82836030363313, 'Fruit': 47.8301709284518, 'légumes': 97.30968772258063}
{'lait': 132.43608686664254, 'oeuf': 136.2966555074753, 'beurre': 90.324468611

In [101]:
#Affichage des résultats
print("Articles en rupture de stock : " + str(Pénurie))
print("Articles jetés : "+ str(Gaspillage))
print("Total des ventes : "+str(Ventes))
print("Gaspillage/commandes : "+  str(100*Gaspillage/(Gaspillage+Ventes)) + " %")
print("Rupture/ventes : "+  str(100*Pénurie/Ventes) +" %")

Articles en rupture de stock : 103740.1181787926
Articles jetés : 30412.395892364526
Total des ventes : 33895810.80544602
Gaspillage/commandes : 0.08964273951709649 %
Rupture/ventes : 0.30605586859755757 %


Scénario 2 : Donnation à une seule organisation 

In [93]:
#instantiation des magasins, des banques alimentaires et des clients 
Magasin = []
for i in range(n):
    Magasin.append(supplyactor.Supermarket(1+VarTM*np.random.random(),"marque"+str(i),i))

for magasin in Magasin : 
    magasin.clients.append(supplyactor.Customer(10*magasin.taille,2*magasin.taille)) #clients
    magasin.BanqueDeDons.append(supplyactor.BanqueAlimentaire(0.3*magasin.taille,0.02*magasin.taille)) #banques alimentaires 

In [94]:
#initialisation des process ARMA pour les clients
for i in range(10*supplyactor.grande_période) : 
    for magasin in Magasin : 
        for client in magasin.clients : 
            client.ARMA()

#initialisation des prédictions pour les magasins
for i in range(10*supplyactor.grande_période) : 
    for magasin in Magasin : 
        magasin.EnregistrementDemande()
        for client in magasin.clients : 
            client.ARMA()
            
#première commande 
for magasin in Magasin : 
    magasin.Commande()


In [95]:
#Simulation
for i in tqdm(range(100*supplyactor.grande_période)):

    supplyactor.temps() #Incrément de temps 

    for magasin in Magasin:
        magasin.Inventaire() #état des stocks
        magasin.Vente() #ventes aux clients des produits disponible en fonction des demandes de leur clients 
        magasin.Dons() #dons aux banques 
        magasin.EnregistrementDemande() #enregistrement de la demande des clients pour les commandes futures 

        for client in magasin.clients : 
            client.ARMA() #actualisation de la demade des clients 

        for banque in magasin.BanqueDeDons : 
            banque.ARMA() #actualisation de la demade des banques alimentaires 

        if i % supplyactor.grande_période == 0:
            magasin.Inventaire() #état des stocks
            magasin.Commande() #commande aux suppliers

In [96]:
#Collecte des résultats

#Denrées perdues 
Gaspillage = 0
for magasin in Magasin : 
    x = magasin.Rebut()
    print(x)
    for product in x.keys() :
        Gaspillage += x[product]

#Demandes non satisfaites 
Pénurie = 0
for magasin in Magasin : 
    Pénurie += magasin.quantitéRupture 

#total des ventes 
Ventes = 0
for magasin in Magasin : 
    Ventes += magasin.ventes

#total des dons
Dons = 0 
for magasin in Magasin : 
    Dons += magasin.dons

#enregistrement
Scénario2 = {"Pénurie" : Pénurie, "Gaspillage":Gaspillage,"Ventes":Ventes, "Dons":Dons}

{'lait': 18.277083265726677, 'oeuf': 301.39180808087167, 'beurre': 123.64411042741165, 'crème': 9.925922761220987, 'fromage': 33.868342627073595, 'Goûter': 143.86101597306595, 'Fruit': 2.4467840788660986, 'légumes': 51.57660564527448}
{'lait': 82.17745399273745, 'oeuf': 129.5540193843052, 'beurre': 63.47942126399913, 'crème': 159.94399514853882, 'fromage': 18.68995036156324, 'Goûter': 20.710983803990896, 'Fruit': 36.71851318989059, 'légumes': 23.593197030812497}
{'lait': 105.59906190103172, 'oeuf': 19.70936916429628, 'beurre': 7.413128267854876, 'crème': 8.026178749558587, 'fromage': 7.064128615645658, 'Goûter': 33.99262759755197, 'Fruit': 27.07463588760958, 'légumes': 46.760518403263674}
{'lait': 13.885263642446759, 'oeuf': 97.6105068458051, 'beurre': 33.01271128903635, 'crème': 34.26474959102417, 'fromage': 56.468692565195454, 'Goûter': 17.624641298247752, 'Fruit': 141.7022206888446, 'légumes': 23.76248484146071}
{'lait': 102.65753325901227, 'oeuf': 32.02269494374558, 'beurre': 22.80

In [97]:
#Affichage des résultats
print("Articles en rupture de stock : " + str(Pénurie))
print("Articles jetés : "+ str(Gaspillage))
print("Articles données : "+ str(Dons))
print("Total des ventes : "+str(Ventes))
print("Gaspillage/commandes : "+  str(100*Gaspillage/(Gaspillage+Ventes)) + " %")
print("Rupture/ventes : "+  str(100*Pénurie/Ventes) +" %")

Articles en rupture de stock : 103740.1181787926
Articles jetés : 30412.395892364526
Articles données : 23734.83719446744
Total des ventes : 33895810.80544602
Gaspillage/commandes : 0.08964273951709649 %
Rupture/ventes : 0.30605586859755757 %


Scénario 3 : donnation à un groupe d'association

In [115]:
#instantiation des magasins, des banques alimentaires et des clients 
Magasin = []
Banques = []
for i in range(n):
    taille = 1+VarTM*np.random.random()
    Magasin.append(supplyactor.Supermarket(taille,"marque"+str(i),i)) #magasin
    Banques.append(supplyactor.BanqueAlimentaire(0.04*taille,0.02*taille)) #banque alimentaire
    
for magasin in Magasin : 
    magasin.clients.append(supplyactor.Customer(10*magasin.taille,2*magasin.taille)) #client
    magasin.BanqueDeDons = Banques #on associe l'enssemble des banques alimentaires à chaque magasin

In [116]:
#initialisation des process ARMA pour les clients
for i in range(10*supplyactor.grande_période) : 
    for magasin in Magasin : 
        for client in magasin.clients : 
            client.ARMA()

#initialisation des prédictions pour les magasins
for i in range(10*supplyactor.grande_période) : 
    for banque in Banques : 
        banque.ARMA()
    for magasin in Magasin : 
        magasin.EnregistrementDemande()
        for client in magasin.clients : 
            client.ARMA()

#première commande
for magasin in Magasin : 
    magasin.Commande()


In [119]:
#Simulation
for i in tqdm(range(100*supplyactor.grande_période)):

    supplyactor.temps() #incrément de temps 

    for banque in Banques : 
        banque.ARMA() #actualisation de la demade des banques alimentaires 

    for magasin in Magasin:
        magasin.Inventaire() #etat des stocks
        magasin.Vente() #ventes aux clients des produits disponible en fonction des demandes de leur clients 
        magasin.Dons() #dons aux banques 
        magasin.EnregistrementDemande() #enregistrement de la demande des clients pour les commandes futures 

        for client in magasin.clients : 
            client.ARMA() #actualisation de la demade des clients
        
        if i % supplyactor.grande_période == 0:
            magasin.Inventaire() #etat des stocks
            magasin.Commande() #commande aux suppliers 

100%|██████████| 1400/1400 [00:16<00:00, 85.40it/s]


In [112]:
#Collecte des résultats

#Denrées perdues 
Gaspillage = 0
for magasin in Magasin : 
    x = magasin.Rebut()
    print(x)
    for product in x.keys() :
        Gaspillage += x[product]

#Demandes non satisfaites 
Pénurie = 0
for magasin in Magasin : 
    Pénurie += magasin.quantitéRupture 

#total des ventes 
Ventes = 0
for magasin in Magasin : 
    Ventes += magasin.ventes

#total des dons
Dons = 0 
for magasin in Magasin : 
    Dons += magasin.dons

#enregistrement
Scénario3 = {"Pénurie" : Pénurie, "Gaspillage":Gaspillage,"Ventes":Ventes, "Dons":Dons}

{'lait': 0, 'oeuf': 0, 'beurre': 0, 'crème': 0, 'fromage': 0, 'Goûter': 0, 'Fruit': 0, 'légumes': 0}
{'lait': 0, 'oeuf': 0, 'beurre': 0, 'crème': 0, 'fromage': 0, 'Goûter': 0, 'Fruit': 0, 'légumes': 0}
{'lait': 0, 'oeuf': 0, 'beurre': 0, 'crème': 0, 'fromage': 0, 'Goûter': 0, 'Fruit': 0, 'légumes': 0}
{'lait': 0, 'oeuf': 0, 'beurre': 0, 'crème': 0, 'fromage': 0, 'Goûter': 0, 'Fruit': 0, 'légumes': 0}
{'lait': 0, 'oeuf': 0, 'beurre': 0, 'crème': 0, 'fromage': 0, 'Goûter': 0, 'Fruit': 0, 'légumes': 0}
{'lait': 0, 'oeuf': 0, 'beurre': 0, 'crème': 0, 'fromage': 0, 'Goûter': 0, 'Fruit': 0, 'légumes': 0}
{'lait': 0, 'oeuf': 0, 'beurre': 0, 'crème': 0.13615646701456852, 'fromage': 0, 'Goûter': 0, 'Fruit': 0, 'légumes': 0}
{'lait': 0, 'oeuf': 0, 'beurre': 0, 'crème': 0, 'fromage': 0, 'Goûter': 0, 'Fruit': 0, 'légumes': 0}
{'lait': 0, 'oeuf': 0, 'beurre': 0, 'crème': 0, 'fromage': 0, 'Goûter': 0.1480875478310415, 'Fruit': 0, 'légumes': 72.44384678331275}
{'lait': 0, 'oeuf': 0, 'beurre': 0, 'crè

In [113]:
#Affichage des résulats 
print("Articles en rupture de stock : " + str(Pénurie))
print("Articles jetés : "+ str(Gaspillage))
print("Articles données : "+ str(Dons))
print("Total des ventes : "+str(Ventes))
print("Gaspillage/commandes : "+  str(100*Gaspillage/(Gaspillage+Ventes)) + " %")
print("Rupture/ventes : "+  str(100*Pénurie/Ventes) +" %")

Articles en rupture de stock : 104645.0668422726
Articles jetés : 8014.193535789511
Articles données : 46328.46921291352
Total des ventes : 29297477.162240148
Gaspillage/commandes : 0.02734707102670695 %
Rupture/ventes : 0.3571811533900389 %


Comparaison des résulats

In [114]:
print("Amélioration dans le scénario2\n")
print("Gaspillage réduit d'un facteur : ",Scénario1["Gaspillage"]/Scénario2["Gaspillage"])
print("Rupture réduit d'un facteur : ",Scénario1["Pénurie"]/Scénario2["Pénurie"])
print("Ventes augmentée d'un facteur",Scénario1["Ventes"]/Scénario2["Ventes"],"\n")
print("Amélioration dans le scénario3\n")
print("Gaspillage réduit d'un facteur : ",Scénario1["Gaspillage"]/Scénario3["Gaspillage"])
print("Rupture réduit d'un facteur : ",Scénario1["Pénurie"]/Scénario3["Pénurie"])
print("Ventes augmentée d'un facteur",Scénario1["Ventes"]/Scénario3["Ventes"])

Amélioration dans le scénario2

Gaspillage réduit d'un facteur :  1.8560430696832533
Rupture réduit d'un facteur :  1.0143976851843426
Ventes augmentée d'un facteur 0.9539073779649553 

Amélioration dans le scénario3

Gaspillage réduit d'un facteur :  7.043343335347319
Rupture réduit d'un facteur :  1.005625386048365
Ventes augmentée d'un facteur 1.1036262211372923
